# Домашнее задание 1. Извлечение ключевых слов

При выполнении домашнего задания можно пользоваться тетрадкой с семинара.

### Описание задания:

1. Подготовить мини-корпус (4-5 текстов или до 10 тысяч токенов) с разметкой ключевых слов.
Желательно указать источник корпуса и описать, в каком виде там были представлены ключевые слова.

2. Разметить ключевые слова самостоятельно. Оценить пересечение с имеющейся разметкой.

3. Применить к этому корпусу 3 метода извлечения ключевых слов на выбор (RAKE, TextRank, tf*idf, OKAPI BM25).

4. Оценить точность, полноту, F-меру выбранных методов относительно имеющейся разметки.

5. Описать ошибки автоматического выделения ключевых слов (что выделяется лишнее, что не выделяется);
предложить свои методы решения этих проблем.

### Критерии оценки:

По 2 балла на каждый пункт.

### Формат сдачи задания:

Jupyter-notebook на гитхабе (запишите адрес своего репозитория [сюда](https://forms.gle/Z4z3JsHbvM6Ghroo9))

### Дедлайн: 

18 ноября 2019 10:00мск

## Скачиваем нужные библиотеки

In [ ]:
import re
import pandas as pd
from nltk.tokenize import RegexpTokenizer
#nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('russian')
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

## Открываем тексты и ключевые слова


In [4]:
file_names = ['business', 'science', 'crime', 'politics']
keyword_file = 'keywords.txt'
texts_dict = {}
for f in file_names:
    fname = f+'.txt'
    with open(fname, 'r', encoding='utf-8') as src:
        text = src.read()
        texts_dict[f] = text

In [173]:
texts_dict

{'business': 'Четырехдневный рабочий день повышает продуктивность компании - к такому выводу пришли в японском филиале корпорации Microsoft, разрешив сотрудникам месяц не работать по пятницам.\n\nКак говорится в заявлении компании, Microsoft Japan каждую пятницу закрывала свои офисы в рамках акции под названием "Work-Life Choice Challenge Summer 2019". Постоянным сотрудникам компании эти выходные оплачивали.\n\nМеры коснулись 90% из 2280 сотрудников компании. Им запретили проводить совещания дольше получаса и рекомендовали решать все рабочие вопросы по мессенджеру, а не в ходе личных встреч.\n\nЖители какой страны работают больше всех? Ответ вас удивит\nЧетырехдневная рабочая неделя: казалось бы, что в этом плохого?\nВ результате производительность, то есть количество продаж на одного сотрудника, выросла почти на 40%.\n\n"Работайте недолго, хорошо отдыхайте и много учитесь. Человеку необходима среда, которая позволяет почувствовать предназначение в жизни и добиваться лучших результатов

#### Лемматизируем:

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
def preproc(text, stopwords, maximal=False):
    stopwords = set(stopwords)
    text = re.sub(r'\n', r' ', text)
    if maximal:
        text = re.sub(r'["%0-9A-Za-z]', r'', text)
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    newtext = ''
    for w in text:
        if w not in stopwords: 
            lemma = morph.parse(w)[0].normal_form + ' '
            newtext += lemma
    return newtext

In [129]:
preproc_texts_dict = {}
for t in texts_dict.keys():
    lemmatized = preproc(texts_dict[t], stop, maximal=False)
    preproc_texts_dict[t] = lemmatized

In [108]:
preproc_texts_dict

{'business': 'четырехдневный рабочий день повышать продуктивность компания такой вывод прислать японский филиал корпорация microsoft разрешить сотрудник месяц работать пятница как говориться заявление компания microsoft japan каждый пятница закрывать свой офис рамка акция название work life choice challenge summer 2019 постоянный сотрудник компания выходной оплачивать мера коснуться 90 2280 сотрудник компания имя запретить проводить совещание долгий получас рекомендовать решать рабочий вопрос мессенджер ход личный встреча житель страна работать ответ удивить четырехдневный рабочий неделя казалось плохой в результат производительность количество продажа один сотрудник вырасти 40 работать недолго отдыхать учиться человек необходимый среда который позволять почувствовать предназначение жизнь добиваться хороший результат работа я хотеть сотрудник стремиться достижение тот результат 20 маленький свободный время сказать президент японский microsoft такуйя хирано это довольно необычный филосо

### И ключевые слова

In [174]:
keywords_dict = {}
keywords_lemmas_dict = {}
with open(keyword_file, 'r', encoding='utf-8') as src:
    kws = src.read()
    kws = kws.split('\n')
    for line in kws:
        text, keywords = (a for a in line.split(': '))
        keywords = keywords.split(', ')
        keywords_dict[text] = keywords
        kw_lemmas = []
        for kw in keywords:
            #kw = kw.split(' ')
            #for k_w in kw:
            lemmas = preproc(kw, stopwords=stop)
            kw_lemmas.append(lemmas[:-1])
        keywords_lemmas_dict[text] = kw_lemmas

In [156]:
keywords_lemmas_dict

{'politics': ['боливия ',
  'государственный переворот ',
  'эво моралеса ',
  'протест ',
  'выбор ',
  'политический убежище '],
 'crime': ['московский дело ',
  'никита чирец ',
  'массовый беспорядок ',
  'выбор мосгордума ',
  'суд '],
 'business': ['четырехдневный рабочий неделя ',
  'япония ',
  'microsoft ',
  'продуктивность '],
 'science': ['кот ',
  'собака ',
  'эмоция ',
  'поведение ',
  'человек ',
  'дистанция ',
  'язык телодвижение ']}

In [158]:
keywords_dict

{'politics': ['Боливия',
  'государственный переворот',
  'Эво Моралес',
  'протесты',
  'выборы',
  'политическое убежище'],
 'crime': ['московское дело',
  'Никита Чирцов',
  'массовые беспорядки',
  'выборы в Мосгордуму',
  'суд'],
 'business': ['четырехдневная рабочая неделя',
  'Япония',
  'Microsoft',
  'продуктивность'],
 'science': ['кот',
  'собака',
  'эмоция',
  'поведение',
  'человек',
  'дистанция',
  'язык телодвижений']}

## Поиск ключевых слов в тексте

Делаем "тупой" поиск по подстрокам. Считаем, сколько получилось вхождений ключевых слов в каждом тексте.

In [167]:
def kw_brute_search(keywords, corpus):
    for name in keywords.keys():
        print(name)
        text = corpus[name]
        kws = keywords[name]
        for kw in kws:
            kw_occs = text.count(kw)
            print(f'{kw} - {kw_occs}')
        print('-----------------')
    return 'done'

#### не-лемматизированные тексты

In [170]:
kw_brute_search(keywords_dict, texts_dict)

politics
Боливия - 2
государственный переворот - 1
Эво Моралес - 3
протесты - 1
выборы - 4
политическое убежище - 2
-----------------
crime
московское дело - 0
Никита Чирцов - 0
массовые беспорядки - 0
выборы в Мосгордуму - 0
суд - 21
-----------------
business
четырехдневная рабочая неделя - 0
Япония - 0
Microsoft - 5
продуктивность - 1
-----------------
science
кот - 31
собака - 6
эмоция - 0
поведение - 2
человек - 8
дистанция - 0
язык телодвижений - 0
-----------------


'done'

не очень...

#### лемматизированные тексты

In [171]:
kw_brute_search(keywords_lemmas_dict, preproc_texts_dict)

politics
боливия - 24
государственный переворот - 4
эво моралеса - 3
протест - 7
выбор - 13
политический убежище - 3
-----------------
crime
московский дело - 2
никита чирец - 0
массовый беспорядок - 1
выбор мосгордума - 0
суд - 23
-----------------
business
четырехдневный рабочий неделя - 1
япония - 1
microsoft - 5
продуктивность - 1
-----------------
science
кот - 32
собака - 21
эмоция - 1
поведение - 4
человек - 15
дистанция - 1
язык телодвижение - 4
-----------------


'done'

Очень неплохо! Особенно когда идет поиск по изолированным ключевым словам.


## Rake

In [177]:
import RAKE

In [178]:
rake = RAKE.Rake(stop)

### Нелемматизированные тексты

In [186]:
for text in texts_dict.keys():
    print(text+':')
    rake_kw_list = rake.run(texts_dict[text], maxWords=3, minFrequency=2)
    for kw in rake_kw_list[:15]:
        print(kw)

business:
('месяц', 1.6666666666666667)
('говорится', 1.0)
('проведенного', 1.0)
science:
('большей части общаются', 9.0)
('языке телодвижений', 4.5)
('подчеркивает витале', 4.5)
('говорит хистэнд', 4.25)
('своим предкам', 4.0)
('говорит', 2.0)
('это', 1.8888888888888888)
('кошки', 1.6875)
('просто', 1.5)
('хозяина', 1.3333333333333333)
('собаки', 1.3)
('собаками', 1.0)
('понимаем', 1.0)
('чувствуют', 1.0)
('судить', 1.0)
crime:
('область грудной клетки', 8.666666666666666)
('снимается вопрос', 3.75)
('суда', 2.0)
('сизо', 1.6666666666666667)
('из-', 1.3333333333333333)
('аквариума', 1.0)
('побежал', 1.0)
politics:
('би-би-си', 9.0)
('тех пор', 4.0)
('это', 2.0)
('боливии', 2.0)
('стране', 1.75)
('отставку', 1.6666666666666667)
('пока', 1.6666666666666667)
('заявил', 1.6666666666666667)
('моралес', 1.5714285714285714)
('однако', 1.3333333333333333)
('отставке', 1.0)
('уйти', 1.0)
('призвали', 1.0)
('беседе', 1.0)
('бразилии', 1.0)


In [182]:
rake_kw_list = rake.run(texts_dict['science'], maxWords=3, minFrequency=4)

In [184]:
wiki_kw_list[:15]

[('большей части общаются', 9.0),
 ('языке телодвижений', 4.5),
 ('подчеркивает витале', 4.5),
 ('говорит хистэнд', 4.25),
 ('своим предкам', 4.0),
 ('говорит', 2.0),
 ('это', 1.8888888888888888),
 ('кошки', 1.6875),
 ('просто', 1.5),
 ('хозяина', 1.3333333333333333),
 ('собаки', 1.3),
 ('собаками', 1.0),
 ('понимаем', 1.0),
 ('чувствуют', 1.0),
 ('судить', 1.0)]

### Лемматизированные тексты

для этого вначале запрепроцессим тексты без стоп-слов. Иначе RAKE не будет работать.

In [189]:
def preproc_rake(text, maximal=False):
    text = re.sub(r'\n', r' ', text)
    if maximal:
        text = re.sub(r'["%0-9A-Za-z]', r'', text)
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    newtext = ''
    for w in text:
        lemma = morph.parse(w)[0].normal_form + ' '
        newtext += lemma
    return newtext

In [190]:
rake_preproc_texts_dict = {}
for t in texts_dict.keys():
    lemmatized = preproc_rake(texts_dict[t], maximal=False)
    rake_preproc_texts_dict[t] = lemmatized

In [192]:
for text in texts_dict.keys():
    print(text+':')
    rake_kw_list = rake.run(preproc_texts_dict[text], maxWords=3, minFrequency=1)
    for kw in rake_kw_list[:15]:
        print(kw)

business:
science:
('смысл совместный эволюция', 9.0)
('явно нравиться компания', 9.0)
('это обязательно вино', 9.0)
('оставить покой', 4.0)
('грамм правда', 4.0)
('вести собака', 4.0)
('навязчивый внимание', 4.0)
('2007 год поехать', 4.0)
('показывать рад', 4.0)
('свой манер', 4.0)
('порядок', 1.0)
crime:
('встретить сергей абаничев', 9.0)
('вопрос примирение рассматривать', 9.0)
('сотрудник полиция предложение', 9.0)
('обсуждать процедура', 4.0)
('часы', 1.0)
politics:
('посольство остаться родина', 9.0)
('россия сотрудничать моралесом', 9.0)
('сменить моралеса', 4.0)
('пора пока', 4.0)
('реакция', 1.0)
('надеяться', 1.0)


## TextRank

In [199]:
from gensim.summarization import keywords as kw

#### Нелемматизированные тексты

In [207]:
from gensim.summarization import keywords as kw
for text in texts_dict.keys():
    print(text+':')
    text_rank = kw(texts_dict[text], pos_filter=[], scores=True)
    for k_w in text_rank[:15]:
        print(f'{k_w[0]}\t{k_w[1]}')
    print('_________________\n\n')

business:
что	0.24671478520666928
microsoft	0.18203806208017073
это	0.1809583099016354
компании	0.1685188413361635
сотрудники	0.14718268167618756
как	0.11745256717674177
сотрудникам	0.10808285749458105
существует	0.1002607957146045
года	0.09567784394788716
страны работают	0.08750775137850704
четырехдневнои	0.08537155766586102
кроме	0.08515591391750012
около	0.08515591391750005
наоборот	0.08515591391750005
количество	0.08515591391749999
_________________


science:
это	0.2235252957955672
или	0.20290370212087455
кошки живут	0.1598507556427199
кажется что	0.15965177043836604
когда они	0.15859588327865745
собаки	0.1229970609573232
вас	0.11730665189698189
есть	0.10039179152418767
она	0.09950847283981586
как животное его	0.09753802514710973
только	0.09460823883672725
так	0.09400131568325784
говорит	0.09034008320299605
телодвижении кошек	0.08880966582184609
животных	0.08835034983846035
_________________


crime:
что	0.24033425994955188
чирцова	0.2340698908697721
как	0.20166717604924841
суд	0.

#### Лемматизированные тексты

In [206]:
for text in texts_dict.keys():
    print(text+':')
    text_rank = kw(preproc_texts_dict[text], pos_filter=[], scores=True)
    for k_w in text_rank[:15]:
        print(f'{k_w[0]}\t{k_w[1]}')
    print('_________________\n\n')

business:
японскии	0.16777632661582675
это	0.1655230437876
microsoft	0.16193307017117922
рабочии день	0.15634711162808876
сотрудник месяц работать пятница как	0.1509487611505443
компания	0.14817323698433021
год существовать	0.13909059064342347
дополнительныи	0.11914303424608577
выходнои	0.11639505900456174
результат	0.11582986390210369
неделя	0.10135961494763161
время сказать	0.1008361959430181
ход	0.09551008135069967
которыи	0.09540672182327103
введение	0.0952325231201877
_________________


science:
кошка гораздо	0.30787619920091946
собака	0.25204747576036024
это	0.2113562983520873
которыи	0.18395172785580727
свои	0.1666208265885797
жить человек	0.1487778489275467
хозяин	0.12632995459065646
кошачии	0.07890713849992222
кот общение	0.07713221607089192
считаться один	0.07691783872897075
место	0.0671753996959183
животное частично	0.06605503274653528
очень	0.06531520407720881
имидж	0.06468919699153869
дать исследование	0.06425529736440051
_________________


crime:
чирцовыи	0.230619425457

## Tf-Idf


#### Нелемматизированные тексты

In [208]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [216]:
corpus = list(texts_dict.values())

In [217]:
cv=CountVectorizer(max_features=10000) #stop_words=stopwords,
word_count_vector=cv.fit_transform(corpus)

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tf_idf_vector=tfidf_transformer.fit_transform(word_count_vector)

df = pd.DataFrame(tf_idf_vector.toarray(), columns=cv.get_feature_names(), index=texts_dict.keys())
df = df.T

In [218]:
col_names = list(df.columns)
#texts_and_keywords = {}
for col in col_names:
    kws = df.nlargest(7, col)
    kws = kws[col]
    print(col)
    print(kws)
    print('___________')


business
на           0.268155
microsoft    0.233574
что          0.195022
компании     0.186859
не           0.146266
месяц        0.140144
недели       0.140144
Name: business, dtype: float64
___________
science
кошки     0.320517
они       0.253039
что       0.228880
на        0.211274
собаки    0.185562
кошек     0.168693
их        0.161512
Name: science, dtype: float64
___________
crime
чирцова    0.293059
не         0.229395
его        0.224467
суд        0.161736
на         0.152930
что        0.152930
адвокат    0.146529
Name: crime, dtype: float64
___________
politics
боливии       0.383843
моралеса      0.365565
моралес       0.237617
на            0.200305
что           0.190767
заявил        0.129697
президента    0.127948
Name: politics, dtype: float64
___________


#### Лемматизированные тексты

In [213]:
corpus = list(preproc_texts_dict.values())

cv=CountVectorizer(max_features=10000) #stop_words=stopwords,
word_count_vector=cv.fit_transform(corpus)

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tf_idf_vector=tfidf_transformer.fit_transform(word_count_vector)

df = pd.DataFrame(tf_idf_vector.toarray(), columns=cv.get_feature_names(), index=texts_dict.keys())
df = df.T

In [215]:
col_names = list(df.columns)
#texts_and_keywords = {}
for col in col_names:
    kws = df.nlargest(7, col)
    kws = kws[col]
    print(col)
    print(kws)
    print('____________')


business
рабочий      0.364951
сотрудник    0.323698
microsoft    0.228094
японский     0.228094
компания     0.179832
неделя       0.143866
работать     0.143866
Name: business, dtype: float64
____________
science
кошка      0.653909
собака     0.334929
хозяин     0.143541
кот        0.127592
который    0.124843
человек    0.124843
это        0.116520
Name: science, dtype: float64
____________
crime
суд            0.318807
чирцовый       0.311051
адвокат        0.186630
боль           0.186630
задержать      0.155525
судья          0.155525
полицейский    0.147142
Name: crime, dtype: float64
____________
politics
моралеса     0.556732
боливия      0.381759
президент    0.200655
выбор        0.174973
отставка     0.174973
страна       0.142142
заявить      0.137951
Name: politics, dtype: float64
____________
